# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846888432404                   -0.70    4.5         
  2   -7.852324674490       -2.26       -1.53    1.0   25.4ms
  3   -7.852615422701       -3.54       -2.56    1.2   27.9ms
  4   -7.852645893107       -4.52       -2.88    2.2   34.2ms
  5   -7.852646498028       -6.22       -3.18    1.2   26.6ms
  6   -7.852646678242       -6.74       -4.02    1.0   24.2ms
  7   -7.852646686272       -8.10       -5.30    1.8   28.6ms
  8   -7.852646686721       -9.35       -5.35    2.0   31.7ms
  9   -7.852646686729      -11.07       -6.01    1.0   27.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846715507501                   -0.70           4.8         
  2   -7.852525082972       -2.24       -1.64   0.80    2.0    247ms
  3   -7.852635074106       -3.96       -2.75   0.80    1.0   23.8ms
  4   -7.852646440309       -4.94       -3.23   0.80    2.2   32.7ms
  5   -7.852646665703       -6.65       -4.02   0.80    1.0   23.6ms
  6   -7.852646686350       -7.69       -4.68   0.80    1.8   27.5ms
  7   -7.852646686702       -9.45       -5.51   0.80    1.2   26.4ms
  8   -7.852646686729      -10.56       -6.69   0.80    2.2   32.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.353930e+01     3.274267e+00
 * time: 0.15859603881835938
     1     9.671937e-01     1.835145e+00
 * time: 0.3876621723175049
     2    -1.922099e+00     2.068739e+00
 * time: 0.412891149520874
     3    -3.947643e+00     1.803159e+00
 * time: 0.4476962089538574
     4    -5.248157e+00     1.558622e+00
 * time: 0.4806079864501953
     5    -6.920804e+00     1.111368e+00
 * time: 0.5155210494995117
     6    -7.508147e+00     5.753841e-01
 * time: 0.5497081279754639
     7    -7.698376e+00     3.082282e-01
 * time: 0.5736911296844482
     8    -7.770219e+00     1.066928e-01
 * time: 0.5978450775146484
     9    -7.809147e+00     1.695387e-01
 * time: 0.6239190101623535
    10    -7.832398e+00     8.359507e-02
 * time: 0.6488721370697021
    11    -7.844114e+00     6.787282e-02
 * time: 0.6736900806427002
    12    -7.848645e+00     4.385409e-02
 * time: 0.7000951766967773
    13    -7.850727e+00     2.151547e-02
 * time: 0.7261471748

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846826093527                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645809054                   -1.64         
  2   -7.852646686730       -6.06       -3.71    2.05s
  3   -7.852646686730      -13.03       -7.15    219ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.1638106218489757e-6
|ρ_newton - ρ_scfv| = 3.507890077019323e-7
|ρ_newton - ρ_dm|   = 1.2831829180515968e-9
